In [1]:
import os
import sys
import glob2 as glob
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.backends.backend_pdf import PdfPages as pdf

In [2]:
# specify which version to plot

# plot_vers = 'first'

plot_vers = 'last'

# specify source files from two model runs

# dir_1 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E2-G/amip/r1i1p1f1/'
dir_1 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/'
# dir_1 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-Omon/historical/r1i1p1f1/'
tim_1 = '2014-09' # specify what month to plot
mod_1 = dir_1.split('/')[-4]
run_1 = dir_1.split('/')[-3]
dir_1_var = glob.glob(dir_1+'/*/*/*', recursive=True) # all reported variables
if dir_1_var == []:
    print('INPUT ERROR: Directory specification error (no variables found).')
    print(dir_1)
    sys.exit()
if mod_1[0:7] == 'GISS-E2': # skip *fx and *fy variables
    dir_1_var = [i for i in dir_1_var if 'fx' not in i]
    dir_1_var = [i for i in dir_1_var if 'fy' not in i]
    print('WARNING: Skipping fx class from E2 owing to dimensionality error.')
ncs_1 = []
for i_1, d_1 in enumerate(dir_1_var):
    v_all = sorted(glob.glob(dir_1_var[i_1]+'/*', recursive=True)) # all versions
    if v_all != []:
        if plot_vers == 'first':
            f_all = sorted(glob.glob(v_all[0]+'/*.nc')) # all files in first version
        elif plot_vers == 'last':
            f_all = sorted(glob.glob(v_all[-1]+'/*.nc')) # or all files in last version
        f_tim = [i for i in f_all if '2014' in i]
        if f_tim == []:
            f_tim = [i for i in f_all if tim_1.replace('-','') in i]
        ncs_1.append(f_tim[0])

# dir_2 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/'
# dir_2 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E2-G/amip/r1i1p1f1/'
dir_2 = '/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/'
tim_2 = '2014-09' # specify what month to plot
mod_2 = dir_2.split('/')[-4]
run_2 = dir_2.split('/')[-3]
dir_2_var = glob.glob(dir_2+'/*/*/*', recursive=True)
if dir_2_var == []:
    print('INPUT ERROR: Directory specification error (no variables found).')
    print(dir_2)
    sys.exit()
if mod_2[0:7] == 'GISS-E2': # skip *fx and *fy variables
    dir_2_var = [i for i in dir_2_var if 'fx' not in i]
    dir_2_var = [i for i in dir_2_var if 'fy' not in i]
    print('WARNING: Skipping fx class from E2 owing to dimensionality error.')
ncs_2 = []
for i_2, d_2 in enumerate(dir_2_var):
    v_all = sorted(glob.glob(dir_2_var[i_2]+'/*', recursive=True))
    if v_all != []:
        if plot_vers == 'first':
            f_all = sorted(glob.glob(v_all[0]+'/*.nc')) # all files in first version
        elif plot_vers == 'last':
            f_all = sorted(glob.glob(v_all[-1]+'/*.nc')) # or all files in last version
        f_tim = [i for i in f_all if '2014' in i]
        if f_tim == []:
            f_tim = [i for i in f_all if tim_1.replace('-','') in i]        
        ncs_2.append(f_tim[0])

# specify local destination for comparison plots

out_dir = './'
out_pdf = out_dir+mod_1+'_'+run_1+'_vs_'+mod_2+'_'+run_2+'.pdf'

In [ ]:
# loop over source files in first model run

pp = pdf('multipage.pdf')

for i_1, f_1 in enumerate(ncs_1):
    print(f_1)
    dat_1 = xr.open_dataset(f_1)
    var_1 = list(dat_1.data_vars.keys())[-1]
    ndims = len(dat_1[var_1].dims)
    if ndims!=2: dat_1 = xr.open_dataset(f_1).sel(time=tim_1) # most fields have time
    # print(dat_1[var_1].dims)
    if ndims==1:
        fig = plt.figure(figsize=[8.5,11])
        ax = fig.add_subplot(211)
        fld_1 = dat_1[var_1].isel(time=0)
        ax.annotate('SCALAR VALUE',xy=(0.4,0.5),xycoords='axes fraction')
        path, fname = os.path.split(f_1)
        parr = path.split(mod_1)
        title = parr[0]+mod_1+'\n'+parr[1]+'/\n'+fname+'\n'+fld_1.attrs['long_name']
        plt.title(title)
        val_str = ("value = "+"{:.5e}".format(fld_1.data))
        ax.annotate(tim_1+' '+val_str,xy=(0,-0.15),xycoords='axes fraction')

        matching_file = [i for i in ncs_2 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
        if matching_file != []:
            f_2 = matching_file[0]
            print(f_2)
            dat_2 = xr.open_dataset(f_2)
            var_2 = list(dat_2.data_vars.keys())[-1]
            ax = fig.add_subplot(212)
            fld_2 = dat_2[var_2].isel(time=0)
            ax.annotate('SCALAR VALUE',xy=(0.4,0.5),xycoords='axes fraction')
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']
            plt.title(title)
            val_str = ("value = "+"{:.5e}".format(fld_2.data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.15),xycoords='axes fraction')
            fig.tight_layout(pad=6)
        
        pp.savefig()        
    elif ndims==2:
        fig = plt.figure(figsize=[8.5,11])
        ax = fig.add_subplot(211,projection=ccrs.PlateCarree(central_longitude=180))
        fld_1 = dat_1[var_1]
        fld_1.plot(ax=ax,transform=ccrs.PlateCarree(),
                    cbar_kwargs={'label': fld_1.units},rasterized=True)
        ax.coastlines()
        path, fname = os.path.split(f_1)
        parr = path.split(mod_1)
        title = parr[0]+mod_1+'\n'+parr[1]+'/\n'+fname+'\n'+fld_1.attrs['long_name']
        plt.title(title)
        val_str = ("min, max, avg = "+"{:.5e}".format(fld_1.min().data)+", "
            "{:.5e}".format(fld_1.max().data)+", "+"{:.5e}".format(fld_1.mean().data))
        ax.annotate(tim_1+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')

        matching_file = [i for i in ncs_2 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
        if matching_file != []:
            f_2 = matching_file[0]
            print(f_2)
            dat_2 = xr.open_dataset(f_2)
            var_2 = list(dat_2.data_vars.keys())[-1]
            ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=180))
            fld_2 = dat_2[var_2]
            fld_2.plot(ax=ax,transform=ccrs.PlateCarree(),
                       cbar_kwargs={'label': fld_2.units},rasterized=True)
            ax.coastlines()
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            fig.tight_layout(pad=6)
            
        pp.savefig()
    elif ndims==3:
        fig = plt.figure(figsize=[8.5,11])
        if dat_1[var_1].dims[1]=='basin':
            ax = fig.add_subplot(211)
            fld_1 = dat_1[var_1].isel(basin=0,time=0)
            subtit = ' (basin=0)'
            fld_1.plot(ax=ax)
        else:
            ax = fig.add_subplot(211,projection=ccrs.PlateCarree(central_longitude=180))
            fld_1 = dat_1[var_1].isel(time=0)
            subtit = ''
            fld_1.plot(ax=ax,transform=ccrs.PlateCarree(),
                       cbar_kwargs={'label': fld_1.units},rasterized=True)
            ax.coastlines()
        path, fname = os.path.split(f_1)
        parr = path.split(mod_1)
        title = parr[0]+mod_1+'\n'+parr[1]+'/\n'+fname+'\n'+fld_1.attrs['long_name']+subtit
        plt.title(title)
        val_str = ("min, max, avg = "+"{:.5e}".format(fld_1.min().data)+", "
            "{:.5e}".format(fld_1.max().data)+", "+"{:.5e}".format(fld_1.mean().data))
        ax.annotate(tim_1+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')

        matching_file = [i for i in ncs_2 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
        if matching_file != []:
            f_2 = matching_file[0]
            print(f_2)
            dat_2 = xr.open_dataset(f_2)
            var_2 = list(dat_2.data_vars.keys())[-1]
            if dat_2[var_2].dims[1]=='basin':
                ax = fig.add_subplot(212)
                fld_2 = dat_2[var_2].isel(basin=0,time=0)
                subtit = ' (basin=0)'
                fld_2.plot(ax=ax)
            else:
                ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=180))
                fld_2 = dat_2[var_2].isel(time=0)
                subtit = ''
                fld_2.plot(ax=ax,transform=ccrs.PlateCarree(),
                           cbar_kwargs={'label': fld_2.units},rasterized=True)
                ax.coastlines()
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            fig.tight_layout(pad=6)

        pp.savefig()
    elif ndims==4:
        fig = plt.figure(figsize=[8.5,11])
        ax = fig.add_subplot(211)
        if dat_1[var_1].dims[1]=='basin':
            fld_1 = dat_1[var_1].isel(basin=0,time=0)
            subtit = ' (basin=0)'
        else:
            fld_1 = dat_1[var_1].isel(lon=0,time=0)
            subtit = ' (lon=0)'
        fld_1.plot(ax=ax,cbar_kwargs={'label': fld_1.units},rasterized=True)
        if dat_1[var_1].dims[1]==('lev') or dat_1[var_1].dims[1]==('plev'): ax.invert_yaxis()
        if dat_1[var_1].dims[2]==('lev'): ax.invert_yaxis() # ocean basin case
        title = fld_1.attrs['long_name'] + ' (Longitude=0)'
        path, fname = os.path.split(f_1)
        parr = path.split(mod_1)
        title = parr[0]+mod_1+'\n'+parr[1]+'/\n'+fname+'\n'+fld_1.attrs['long_name']+subtit
        plt.title(title)
        val_str = ("min, max, mean = "+"{:.5e}".format(fld_1.min().data)+", "
            "{:.5e}".format(fld_1.max().data)+", "+"{:.5e}".format(fld_1.mean().data))
        ax.annotate(tim_1+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
        
        matching_file = [i for i in ncs_2 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
        if matching_file != []:
            f_2 = matching_file[0]
            # print(f_2)
            dat_2 = xr.open_dataset(f_2)
            var_2 = list(dat_2.data_vars.keys())[-1]
            ax = fig.add_subplot(212)
            if dat_2[var_2].dims[1]=='basin':
                fld_2 = dat_2[var_2].isel(basin=0,time=0)
                subtit = ' (basin=0)'
            else:
                fld_2 = dat_1[var_2].isel(lon=0,time=0)
                subtit = ' (lon=0)'
            fld_2.plot(ax=ax,cbar_kwargs={'label': fld_2.units},rasterized=True)
            if dat_2[var_2].dims[1]==('lev') or dat_2[var_2].dims[1]==('plev'): ax.invert_yaxis()
            if dat_2[var_2].dims[2]==('lev'): ax.invert_yaxis() # ocean basin case
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            plt.title(title)
            fig.tight_layout(pad=6)

        pp.savefig()
    else: # more than 4 dimensions: also choose a latitude
        fig = plt.figure(figsize=[8.5,11])
        ax = fig.add_subplot(211)
        fld_1 = dat_1[var_1].isel(lat=0,lon=0,time=0)
        subtit = ' (Lat/Lon=0/0)'
        fld_1.plot(ax=ax,cbar_kwargs={'label': fld_1.units},rasterized=True)
        title = fld_1.attrs['long_name'] + subtit
        path, fname = os.path.split(f_1)
        parr = path.split(mod_1)
        title = parr[0]+mod_1+'\n'+parr[1]+'/\n'+fname+'\n'+fld_1.attrs['long_name']+subtit
        plt.title(title)
        val_str = ("min, max, mean = "+"{:.5e}".format(fld_1.min().data)+", "
            "{:.5e}".format(fld_1.max().data)+", "+"{:.5e}".format(fld_1.mean().data))
        ax.annotate(tim_1+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
        
        matching_file = [i for i in ncs_2 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
        if matching_file != []:
            f_2 = matching_file[0]
            # print(f_2)
            dat_2 = xr.open_dataset(f_2)
            var_2 = list(dat_2.data_vars.keys())[-1]
            ax = fig.add_subplot(212)
            fld_2 = dat_1[var_2].isel(lat=0,lon=0,time=0)
            subtit = ' (Lat/Lon=0/0)'
            fld_2.plot(ax=ax,cbar_kwargs={'label': fld_2.units},rasterized=True)
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            plt.title(title)
            fig.tight_layout(pad=6)

        pp.savefig()
        
    # if i_1 == 0: print(stop_me)
    plt.close()

# loop over source files in second model run (plot only any missing from first run)

for i_2, f_2 in enumerate(ncs_2):
    path, fname = os.path.split(f_2)
    matching_file = [i for i in ncs_1 if fname.split('_')[0]+'_'+fname.split('_')[1] in i]
    if matching_file == []:
        print(f_2)
        dat_2 = xr.open_dataset(f_2)
        var_2 = list(dat_2.data_vars.keys())[-1]
        ndims = len(dat_2[var_2].dims)
        if ndims!=2: dat_2 = xr.open_dataset(f_2).sel(time=tim_2) # usually time is a dimension
        if ndims==1:
            fig = plt.figure(figsize=[8.5,11])
            ax = fig.add_subplot(212)
            fld_2 = dat_2[var_2].isel(time=0)
            ax.annotate('SCALAR VALUE',xy=(0.4,0.5),xycoords='axes fraction')
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']
            plt.title(title)
            val_str = ("value = "+"{:.5e}".format(fld_2.data))
            ax.annotate(tim_1+' '+val_str,xy=(0,-0.15),xycoords='axes fraction')
            pp.savefig()        
        elif ndims==2:
            fig = plt.figure(figsize=[8.5,11])
            ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=180))
            fld_2 = dat_2[var_2]
            fld_2.plot(ax=ax,transform=ccrs.PlateCarree(),
                       cbar_kwargs={'label': fld_2.units},rasterized=True)
            ax.coastlines()
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            plt.title(title)
            pp.savefig()
        elif ndims==3:
            fig = plt.figure(figsize=[8.5,11])
            if dat_2[var_2].dims[1]=='basin':
                ax = fig.add_subplot(212)
                fld_2 = dat_2[var_2].isel(basin=0,time=0)
                subtit = ' (basin=0)'
                fld_2.plot(ax=ax)
            else:
                ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=180))
                fld_2 = dat_2[var_2].isel(time=0)
                subtit = ''
                fld_2.plot(ax=ax,transform=ccrs.PlateCarree(),
                           cbar_kwargs={'label': fld_2.units},rasterized=True)
                ax.coastlines()
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            plt.title(title)
            pp.savefig()
        elif ndims==4:
            fig = plt.figure(figsize=[8.5,11])
            ax = fig.add_subplot(212)
            if dat_2[var_2].dims[1]=='basin':
                fld_2 = dat_2[var_2].isel(basin=0,time=0)
                subtit = ' (basin=0)'
            else:
                fld_2 = dat_2[var_2].isel(lon=0,time=0)
                subtit = ' (lon=0)'
            fld_2.plot(ax=ax,cbar_kwargs={'label': fld_2.units},rasterized=True)
            if dat_2[var_2].dims[1]==('lev') or dat_2[var_2].dims[1]==('plev'): ax.invert_yaxis()
            if dat_2[var_2].dims[2]==('lev'): ax.invert_yaxis() # ocean basin case
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            pp.savefig()
        else:
            fig = plt.figure(figsize=[8.5,11])
            ax = fig.add_subplot(212)
            fld_2 = dat_1[var_2].isel(lat=0,lon=0,time=0)
            subtit = ' (Lat/Lon=0/0)'
            fld_2.plot(ax=ax,cbar_kwargs={'label': fld_2.units},rasterized=True)
            path, fname = os.path.split(f_2)
            parr = path.split(mod_2)
            title = parr[0]+mod_2+'\n'+parr[1]+'/\n'+fname+'\n'+fld_2.attrs['long_name']+subtit
            plt.title(title)
            val_str = ("min, max, mean = "+"{:.5e}".format(fld_2.min().data)+", "
                "{:.5e}".format(fld_2.max().data)+", "+"{:.5e}".format(fld_2.mean().data))
            ax.annotate(tim_2+' '+val_str,xy=(0,-0.25),xycoords='axes fraction')
            pp.savefig()
            
        plt.close()
        
pp.close()
os.popen('mv multipage.pdf '+out_pdf)

/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/msftbarot/gn/v20210505/msftbarot_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/msftbarot/gn/v20210505/msftbarot_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc


<ipython-input-3-86f386562649>:125: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout(pad=6)


/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/vo/gn/v20210505/vo_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/uo/gn/v20210505/uo_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/msftmzmpa/gn/v20210505/msftmzmpa_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/mlotst/gn/v20210505/mlotst_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/mlotst/gn/v20210505/mlotst_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc


<ipython-input-3-86f386562649>:125: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout(pad=6)


/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/hfy/gn/v20210505/hfy_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Omon/hfy/gn/v20210505/hfy_Omon_GISS-E3-G_historical_r1i1p1f1_gn_201409-201409.nc


...

<ipython-input-3-86f386562649>:125: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout(pad=6)


/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Lmon/mrso/gr/v20210505/mrso_Lmon_GISS-E3-G_historical_r1i1p1f1_gr_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Lmon/mrso/gr/v20210505/mrso_Lmon_GISS-E3-G_historical_r1i1p1f1_gr_201409-201409.nc


<ipython-input-3-86f386562649>:125: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout(pad=6)


/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Lmon/mrsos/gr/v20210505/mrsos_Lmon_GISS-E3-G_historical_r1i1p1f1_gr_201409-201409.nc
/Users/afridlin/data/CMIP6/CMIP/NASA-GISS/GISS-E3-G-2014/historical/r1i1p1f1/Lmon/mrsos/gr/v20210505/mrsos_Lmon_GISS-E3-G_historical_r1i1p1f1_gr_201409-201409.nc


<ipython-input-3-86f386562649>:125: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  fig.tight_layout(pad=6)
